In [1]:
import json
import time
from pathlib import Path
from schema_mapping_validator import process_combinations
from fragility_mapping_checker import count_model_ids, check_fragility_mapping

# Define paths to required files

This step defines the necessary input files for the schema mapping validation process and creates an output directory
where all results will be saved. Ensure that the paths to the input files (mapping script, JSON schema, and fragility
CSV) are correctly set before running the notebook.

In [2]:
mapping_script = "mapping_modules/HAZUS_EQ/mapping_IM.py"
mapping_function = "auto_populate"
json_schema_file = "mapping_modules/HAZUS_EQ/input_schema.json"
fragility_csv = "fragility_database/fragility.csv"

output_dir = Path("output_files")

In [3]:
output_dir.mkdir(exist_ok=True) # Creates the directory to store output files

# Process combinations and validate results

This step uses the `process_combinations` function from the `schema_mapping_validator` module to generate and validate
combinations derived from the provided JSON schema. Valid and invalid results are saved in the output directory.

In [4]:
start_time = time.time()
print("\nProcessing combinations and validating results...")
results = process_combinations(
    mapping_script, mapping_function, json_schema_file, use_multiprocessing=True
)

valid_results_file = output_dir / "valid_combinations.json"
invalid_results_file = output_dir / "invalid_combinations.json"

with open(valid_results_file, "w") as valid_file:
    json.dump(results["valid"], valid_file, indent=4)

with open(invalid_results_file, "w") as invalid_file:
    json.dump(results["invalid"], invalid_file, indent=4)

print("Summary:")
print(f"Total combinations: {len(results['valid']) + len(results['invalid'])}")
print(f"Valid combinations: {len(results['valid'])}")
print(f"Invalid combinations: {len(results['invalid'])}")
print(f"Step completed in {time.time() - start_time:.2f} seconds")


Processing combinations and validating results...
Summary:
Total combinations: 21504
Valid combinations: 13440
Invalid combinations: 8064
Step completed in 37.31 seconds


# Count Model IDs from Valid Results

This step extracts unique model IDs from the valid results using the `count_model_ids` function from the
`fragility_mapping_checker` module. A summary of these counts is saved in the output directory.

In [5]:
start_time = time.time()
print("\nCounting model IDs from valid results...")
model_id_counts_file = output_dir / "model_id_counts.json"
model_id_counts = count_model_ids(results["valid"], model_id_counts_file)

print(f"Total unique model IDs: {len(model_id_counts)}")
print(f"Step completed in {time.time() - start_time:.2f} seconds")


Counting model IDs from valid results...
Total unique model IDs: 120
Step completed in 0.01 seconds


# Check Fragility Mapping

This step verifies whether the fragility model IDs in the provided database have been mapped to by the valid combinations.
The results are saved in the output directory, and a summary is displayed, including a list of unmapped IDs.

In [6]:
start_time = time.time()
print("\nChecking fragility mapping...")
fragility_mapping_output_file = output_dir / "fragility_mapping_summary.json"
fragility_summary = check_fragility_mapping(results["valid"], fragility_csv, fragility_mapping_output_file)

print("\nFragility Mapping Summary:")
print(f"Total # fragility model IDs: {fragility_summary['total_fragility_ids']}")
print(f"# mapped fragility model IDs: {fragility_summary['mapped']}")
print(f"# unmapped fragility model IDs: {fragility_summary['unmapped']}")
print("\nUnmapped fragility model IDs:")
for unmapped_id in fragility_summary["unmapped_ids"]:
    print(unmapped_id)
print(f"Step completed in {time.time() - start_time:.2f} seconds")


Checking fragility mapping...

Fragility Mapping Summary:
Total # fragility model IDs: 132
# mapped fragility model IDs: 108
# unmapped fragility model IDs: 24

Unmapped fragility model IDs:
LF.W1.HC
LF.W1.MC
LF.W1.LC
LF.W1.PC
LF.W2.HC
LF.W2.MC
LF.W2.LC
LF.W2.PC
LF.S3.HC
LF.S3.MC
LF.S3.LC
LF.S3.PC
LF.PC1.HC
LF.PC1.MC
LF.PC1.LC
LF.PC1.PC
LF.MH.HC
LF.MH.MC
LF.MH.LC
LF.MH.PC
GF.H.S
GF.H.D
GF.V.S
GF.V.D
Step completed in 0.01 seconds


In [7]:
# Display valid combinations without a fragility model ID
print("\nValid results not mapped to a fragility model ID:")
unmapped_model_ids = set(model_id_counts.keys()) - set(fragility_summary["mapped_ids"])
unmapped_model_ids_file = output_dir / "unmapped_model_ids.json"

with open(unmapped_model_ids_file, "w") as unmapped_file:
    json.dump(list(unmapped_model_ids), unmapped_file, indent=4)

for model_id in unmapped_model_ids:
    print(model_id)


Valid results not mapped to a fragility model ID:
LF.PC1.L.MC
LF.PC1.H.MC
LF.PC1.L.LC
LF.PC1.M.LC
LF.PC1.M.HC
LF.PC1.H.HC
LF.PC1.M.MC
LF.PC1.H.PC
LF.PC1.L.HC
LF.PC1.H.LC
LF.PC1.L.PC
LF.PC1.M.PC
